In [1]:
import evaluate
import numpy as np
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import (AutoModelForTokenClassification, AutoTokenizer,
                          DataCollatorForTokenClassification, Trainer,
                          TrainingArguments)

from utils import id2label, label2id, label2id_PRUNED

In [2]:
base_url = "https://huggingface.co/datasets/Babelscape/multinerd/resolve/main/"
dataset = load_dataset("json", data_files={"train": base_url + "train/train_en.jsonl", "validation": base_url + "val/val_en.jsonl", "test": base_url + "test/test_en.jsonl"})
dataset
print(dataset["train"][0])

{'tokens': ['The', 'type', 'locality', 'is', 'Kīlauea', '.'], 'ner_tags': [0, 0, 0, 0, 5, 0], 'lang': 'en'}


In [3]:

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [4]:
example = dataset["train"][0]
print(example)
tokenized_example = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_example["input_ids"])
print(tokens)

{'tokens': ['The', 'type', 'locality', 'is', 'Kīlauea', '.'], 'ner_tags': [0, 0, 0, 0, 5, 0], 'lang': 'en'}
['[CLS]', 'the', 'type', 'locality', 'is', 'ki', '##lau', '##ea', '.', '[SEP]']


In [5]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for idx, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx != previous_word_idx and word_idx is not None:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [6]:
tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched = True
)

In [7]:
tokenized_dataset["train"][0]

{'tokens': ['The', 'type', 'locality', 'is', 'Kīlauea', '.'],
 'ner_tags': [0, 0, 0, 0, 5, 0],
 'lang': 'en',
 'input_ids': [101, 1996, 2828, 10246, 2003, 11382, 17298, 5243, 1012, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 0, 0, 0, 0, 5, -100, -100, 0, -100]}

In [8]:

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [9]:

seqeval = evaluate.load("seqeval")

In [10]:

labels = [id2label[i] for i in example[f"ner_tags"]]

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
}

In [11]:

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels = 31,
    id2label = id2label,
    label2id = label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
notebook_login()

In [13]:

training_args = TrainingArguments(
    output_dir= "my_ner_test",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.005,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to=["none"]
)

In [14]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    args=training_args
)

In [15]:
trainer.train()

In [16]:
trainer.evaluate(tokenized_dataset['test'])

Part 2: Remove all classes except PERSON(PER), ORGANISATION(ORG), LOCATION(LOC), DISEASES(DIS)

In [17]:
k = np.array(list(id2label.keys()))
v = np.array(list(label2id_PRUNED.values()))

mapping_ar = np.zeros(k.max()+1,dtype=v.dtype)
mapping_ar[k] = v
mapping_ar[[10,11,12,13,14,15,16,17,18,19,20]]

array([ 0,  0,  0, 13, 14,  0,  0,  0,  0,  0,  0])

In [18]:

print(id2label.keys())
print(dataset["train"]["ner_tags"][1:20])

def remove_categories(dataset):
    for i, ner_tag in enumerate(dataset["ner_tags"]):
        dataset["ner_tags"][i] = mapping_ar[ner_tag]
    return dataset

dataset = dataset.map(remove_categories)
print(dataset["train"]["ner_tags"][1:20])

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 26, 0, 0, 0, 0, 0, 0, 0, 0, 25, 26, 0, 0, 0], [0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 2, 0], [0, 0, 1, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 14, 14, 0], [0, 0, 0, 0, 0, 0, 5, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 5, 0], [0, 0, 0, 25, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 21, 22, 22, 22, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [19]:
tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched = True
)


trainer = Trainer(
    model = model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    args=training_args
)

Map:   0%|          | 0/131280 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(tokenized_dataset['test'])